In [7]:
using Random
using Distributions
using LinearAlgebra
using DataFrames
using StatsBase
using PDMats
using CSV
using Dates
using JuMP
using Ipopt
import SpecialFunctions: loggamma
import MathOptInterface as MOI

# Seed (student number)
Random.seed!(23249534)

TaskLocalRNG()

In [8]:
const EPS = 1e-8
safe_beta(α, β) = Beta(max(α, EPS), max(β, EPS))

const LOG_EPS = 1e-200
safe_log(x) = log(max(x, LOG_EPS))

const GAMMA_EPS = 1e-8
function safe_gamma(shape, rate)
    α = max(shape, GAMMA_EPS)
    r = max(rate,  GAMMA_EPS)
    θ = 1/r
    θ = max(θ, GAMMA_EPS)
    return Gamma(α, θ)
end

safe_bernoulli(p::Real) = Bernoulli(clamp(p, 0.0, 1.0))

function safe_pdmat(A::AbstractMatrix{<:Real}; eps0=1e-8, maxiter=5)
    S = Symmetric((A + A')/2)
    for i in 0:maxiter
        try
            return PDMat(Float64.(S + (eps0*i)*I))
        catch e
            if !(e isa PosDefException) || i==maxiter
                rethrow(e)
            end
        end
    end
end

const DIR_EPS = 1e-8
function safe_dirichlet(α::AbstractVector{<:Real})
    a = max.(Float64.(α), DIR_EPS)
    s = sum(a)
    x = rand(Dirichlet(a))
    if any(!isfinite, x) || any(<(0.0), x)
        return a ./ s
    end
    sx = sum(x)
    return (isfinite(sx) && abs(sx - 1.0) ≤ 1e-10) ? x : (a ./ s)
end

safe_dirichlet (generic function with 1 method)

In [9]:
# Load the CSV
df = CSV.read("Y_full.csv", DataFrame)
select!(df, Not(1))

# Convert to a Matrix{Float64}
dropmissing!(df)
Y_full = Matrix{Float64}(df)

# Split into training and test data
lookback = 504
Y      = copy(Y_full[1:lookback, :])
Y_test = copy(Y_full[lookback+1:end, :])

2515×10 Matrix{Float64}:
  0.002772     0.00304377   -0.001582     …  -0.00306701  -0.00941998
 -0.0315148   -0.0270803    -0.0225526       -0.0164713   -0.0302804
 -0.035205    -0.0195559    -0.0106637       -0.00201776  -0.00438688
  0.0092706   -0.000122558   0.00724852       0.0102103    0.000777582
  0.0150479    0.0121157     0.0239677        0.0477334    0.0132087
 -0.0223336   -0.0178959    -0.012284     …  -0.00936028  -0.00766867
 -0.00787696  -0.0390295    -0.00544479      -0.0112801   -0.00386406
 -0.00714577  -0.00345951   -0.000746576      0.00507069   0.0100857
 -0.0193923    0.00784288   -0.00485605       0.00252242  -0.00716875
 -0.037105    -0.0224522    -0.00112633       0.010936     0.00489949
  ⋮                                       ⋱               
 -0.00517546  -0.0238215    -0.0109677       -0.0260381   -0.00455082
 -0.0421823   -0.0410992    -0.0196799        0.029247    -0.010888
  0.00440396  -0.0112511     0.0127459       -0.0209517   -0.0100346
  0.0112541

In [10]:
# Warm-start container for run_pp
Base.@kwdef mutable struct PPState
    K::Int
    S::Vector{Int}
    γ::Float64
    β::Vector{Float64}
    ξ::Vector{Float64}
    χ::Vector{Float64}
    α::Vector{Float64}
    λ::Vector{Float64}
    π_vec::Vector{Vector{Float64}}
    μ::Vector{Vector{Float64}}
    Σ::Vector{PDMat{Float64}}
    τ::Vector{Float64}
    ι::Float64
    ν::Float64
end


PPState

In [11]:
function run_pp(Y::Matrix{Float64}, T::Int, H::Int, D::Int; state::Union{Nothing,PPState}=nothing)  
    total_prop     = 0
    total_accepted = 0 

    # Stick‐breaking and concentration priors
    G1, G2 = 1, 5
    C1, C2 = 3, 1
    X1, X2 = 3, 2
    I1, I2 = 1.5, 0.75
    mh_walk = 0.5

    # Initial number of clusters
    K_init   = 4

    # NIW prior hyperparameters
    mu0     = zeros(D)
    kappa0  = 5
    rho0    = D + 2
    Psi0 = safe_pdmat(0.0005 * Matrix{Float64}(I, D, D))
    
    if state === nothing
        N_warmup  = 1000
        N_sample  = 500

        K = K_init
        S = rand(1:K, T)
    
        # Draw γ and β
        γ = rand(safe_gamma(G1, G2))
        β = safe_dirichlet(vcat(ones(K), γ));   β .= β ./ sum(β)
    
        # Preallocate containers
        ξ = Vector{Float64}(undef, K+1)
        χ = Vector{Float64}(undef, K+1)
        α = Vector{Float64}(undef, K+1)
        λ = Vector{Float64}(undef, K+1)
        π_vec = Vector{Vector{Float64}}(undef, K+1)
        μ = Vector{Vector{Float64}}(undef, K+1)
        Σ = Vector{PDMat{Float64}}(undef, K+1)
        τ = Vector{Float64}(undef, T)
    
        δ = PDMat(Matrix{Float64}(I, K+1, K+1))

        # Loop over components j = 1:(K+1)
        for j in 1:K+1
            # Draw stick‐breaking parameters
            ξ[j] = rand(safe_beta(X1, X2))
            χ[j] = rand(safe_gamma(C1, C2))

            # Compute concentration parameters
            α[j] = (1 - ξ[j]) * χ[j]
            λ[j] =        ξ[j]  * χ[j]

            # Build the Dirichlet parameter vector for π_j
            δ_j = δ[:, j]
            dir_pars = α[j] .* β .+ λ[j] .* δ_j
            π_vec[j] = safe_dirichlet(dir_pars)
            π_vec[j] .= π_vec[j] ./ sum(π_vec[j])

            # Draw from NIW:
            Σ_mat = rand(InverseWishart(rho0, Psi0))
            Σ[j]  = safe_pdmat(Σ_mat)
            μ[j]  = rand(MvNormal(mu0, Σ[j] / kappa0))
        end
        ι = rand(Normal(I1, I2))
        ν = 2 + exp(ι)

        # Count assignments and draw latent precision
        m     = zeros(Int, K+1)
        n_mat = zeros(Int, K+1, K+1)
        q_mat = zeros(Int, K, K)
        Q     = zeros(Int, K)

        m[S[1]] += 1
        τ[1] = rand(safe_gamma(ν/2, ν/2))
        for t in 2:T
            m[S[t]]             += 1
            n_mat[S[t-1], S[t]] += 1
            τ[t] = rand(safe_gamma(ν/2, ν/2))
        end

        for s in 1:K
            if m[s] == 0
                # Decrement cluster count
                K -= 1

                # Relabel all states > s
                S[S .> s] .-= 1

                # Remove the s-th entry from each 1‑D vector
                deleteat!(β,  s)
                deleteat!(χ,  s)
                deleteat!(ξ,  s)
                deleteat!(α,  s)
                deleteat!(λ,  s)
                deleteat!(μ,  s)
                deleteat!(Σ,  s)
                deleteat!(m,  s)
                deleteat!(Q,  s)

                # Prune the square matrix
                idx     = [i for i in 1:size(n_mat,1) if i != s]
                n_mat   = n_mat[idx, idx]
                π_vec = [π_vec[i][idx] for i in idx]
                idx     = [i for i in 1:size(q_mat,1) if i != s]
                q_mat   = q_mat[idx, idx]
                δ = PDMat(Matrix{Float64}(I, K+1, K+1))

                β[K+1] = 1.0 - sum(β[1:K])
                β .= β ./ sum(β)
                for j in 1:K+1
                    π_vec[j][K+1] = 1.0 - sum(π_vec[j][1:K])
                    π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
                end
            end
        end
    else
        N_warmup  = 100
        N_sample  = 500

        # Re-use the previous iteration’s variables
        K     = state.K
        S     = copy(state.S)
        γ     = state.γ
        β     = copy(state.β)
        ξ     = copy(state.ξ)
        χ     = copy(state.χ)
        α     = copy(state.α)
        λ     = copy(state.λ)
        π_vec = deepcopy(state.π_vec)
        μ     = deepcopy(state.μ)
        Σ     = deepcopy(state.Σ)
        τ     = deepcopy(state.τ)
        ι     = state.ι
        ν     = state.ν
    
        δ = PDMat(Matrix{Float64}(I, K+1, K+1))

        # Shift left by 1
        S[1:end-1] = S[2:end]

        # Sample the new S[T] from π[S[T-1]]
        s_prev = S[end-1]
        w = copy(π_vec[s_prev][1:K])
        sw = sum(w)
        if sw <= 0 || !isfinite(sw)
            fill!(w, 1.0 / K)
        else
            @. w = w / sw
        end

        # Draw S[T] ~ Categorical(w)
        S[end] = rand(Categorical(w))

        # Draw τ[T]
        τ[1:end-1] = τ[2:end]
        τ[end] = rand(safe_gamma(ν/2, ν/2))

        # Count assignments
        m     = zeros(Int, K+1)
        n_mat = zeros(Int, K+1, K+1)
        q_mat = zeros(Int, K, K)
        Q     = zeros(Int, K)

        m[S[1]] += 1
        for t in 2:T
            m[S[t]]             += 1
            n_mat[S[t-1], S[t]] += 1
        end

        dead = findall(==(0), m[1:K])
        if !isempty(dead)
            for s in sort(dead; rev=true)
                # Relabel states > s
                S[S .> s] .-= 1
                # 1-D vectors
                deleteat!(β, s); deleteat!(χ, s); deleteat!(ξ, s)
                deleteat!(α, s); deleteat!(λ, s); deleteat!(μ, s)
                deleteat!(Σ, s); deleteat!(m, s); deleteat!(Q, s)
                # Square matrices
                idx = [i for i in 1:size(n_mat,1) if i != s]
                n_mat = n_mat[idx, idx]
                # Rows of π_vec and truncate their lengths
                π_vec = [π_vec[i] for i in idx]
                for i in 1:K
                    π_vec[i] = [π_vec[i][j] for j in idx]
                end    
                idx = [i for i in 1:size(q_mat,1) if i != s]
                q_mat = q_mat[idx, idx]
                K -= 1
            end
            δ = PDMat(Matrix{Float64}(I, K+1, K+1))
            # renormalise β and π rows for new K
            β[K+1] = 1.0 - sum(β[1:K]); β .= β ./ sum(β)
            for j in 1:K+1
                π_vec[j][K+1] = 1.0 - sum(π_vec[j][1:K])
                π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
            end
        end
    end

    # Total number of iterations
    total_iters = N_warmup + N_sample

    # Preallocate prediction containers
    mu_bar      = zeros(Float64, H, D)
    mumuT_bar   = zeros(Float64, H, D, D)
    Sigma_bar   = zeros(Float64, H, D, D)
    M_bar       = zeros(Float64, H, D, D)

    for n_iter in 1:total_iters
        # Update latent states & counts
        for t in 1:T
            # Decrement counts for the current state
            m[S[t]] -= 1
            if t > 1
                n_mat[S[t-1], S[t]] -= 1
            end
            if t < T
                n_mat[S[t], S[t+1]] -= 1
            end
            logW = fill(-Inf, length(β))
            for k in 1:length(β)
                if t == 1
                    log_w_prev = safe_log(β[k]) + safe_log(π_vec[k][S[t+1]])
                elseif t == T
                    log_w_prev = safe_log(π_vec[S[t-1]][k])
                else
                    log_w_prev = safe_log(π_vec[S[t-1]][k]) + safe_log(π_vec[k][S[t+1]])
                end
                if isfinite(log_w_prev)
                    resid = Y[t, :] .- μ[k]
                    quad = dot(resid, (Σ[k] / τ[t]) \ resid)
                    logW[k] = log_w_prev - 0.5 * (quad + logdet(Σ[k] / τ[t]))
                end
            end
            # Stabilise and exponentiate
            maxLW = maximum(logW)
            if !isfinite(maxLW)
                P = β
            else
                W = exp.(logW .- maxLW)
                P = W ./ sum(W)
            end

            # Resample the state
            S[t] = rand(Categorical(P))
            m[S[t]] += 1
            if t > 1
                n_mat[S[t-1], S[t]] += 1
            end
            if t < T
                n_mat[S[t], S[t+1]] += 1
            end

            # Update τ
            resid = Y[t, :] .- μ[S[t]]
            quad = dot(resid, Σ[S[t]] \ resid)
            τ[t] = rand(safe_gamma((ν + D)/2, (ν + quad)/2))

            if S[t] == K+1
                K += 1

                push!(β,   NaN)
                push!(χ,   NaN)
                push!(ξ,   NaN)
                push!(α,   NaN)
                push!(λ,   NaN)
                push!(μ,   fill(NaN, D))
                push!(Σ,   safe_pdmat(zeros(D,D)))
                push!(m,     0)
                push!(Q,     0)

                δ = PDMat(Matrix{Float64}(I, K+1, K+1))

                # extend the transition‐count matrix
                n_mat = vcat(n_mat, zeros(Int, 1, size(n_mat, 2)))
                n_mat = hcat(n_mat, zeros(Int, size(n_mat, 1), 1))

                q_mat = vcat(q_mat, zeros(Int, 1, size(q_mat, 2)))
                q_mat = hcat(q_mat, zeros(Int, size(q_mat, 1), 1))

                stick  = rand(safe_beta(1, γ))
                β[K+1] = β[K] * (1 - stick)
                β[K]   = β[K] * stick
                β .= β ./ sum(β)

                ξ[K+1] = rand(safe_beta(X1, X2))
                χ[K+1] = rand(safe_gamma(C1, C2))
            
                # Compute concentration parameters
                α[K+1] = (1 - ξ[K+1]) * χ[K+1]
                λ[K+1] =      ξ[K+1]  * χ[K+1]

                # Draw from NIW
                Σ_mat = rand(InverseWishart(rho0, Psi0))
                Σ[K+1]  = safe_pdmat(Σ_mat)
                μ[K+1]  = rand(MvNormal(mu0, Σ[K+1] / kappa0))

                # Expand each existing row‐vector to length K+1
                for row in π_vec
                    push!(row, 0.0)
                end

                # Append the new (K+1)th row
                push!(
                    π_vec,
                    safe_dirichlet(α[K+1] .* β .+ λ[K+1] .* δ[:, K+1])
                )
                π_vec[K+1] .= π_vec[K+1] ./ sum(π_vec[K+1])

                # Do the stick‐breaking update for rows 1:K
                for j in 1:K
                    wK = π_vec[j][K]
                    a  = α[j] * β[K]   + λ[j] * δ[j, K]
                    b  = α[j] * β[K+1]
                    stick = rand(safe_beta(a, b))
                    π_vec[j][K]   = stick * wK
                    π_vec[j][K+1] = (1 - stick) * wK
                    π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
                end
            end
        end

        dead = findall(==(0), m[1:K])
        if !isempty(dead)
            for s in sort(dead; rev=true)
                # Relabel states > s
                S[S .> s] .-= 1
                # 1-D vectors
                deleteat!(β, s); deleteat!(χ, s); deleteat!(ξ, s)
                deleteat!(α, s); deleteat!(λ, s); deleteat!(μ, s)
                deleteat!(Σ, s); deleteat!(m, s); deleteat!(Q, s)
                # Square matrices
                idx = [i for i in 1:size(n_mat,1) if i != s]
                n_mat = n_mat[idx, idx]
                # Rows of π_vec and truncate their lengths
                π_vec = [π_vec[i] for i in idx]
                for i in 1:K
                    π_vec[i] = [π_vec[i][j] for j in idx]
                end    
                idx = [i for i in 1:size(q_mat,1) if i != s]
                q_mat = q_mat[idx, idx]
                K -= 1
            end
            δ = PDMat(Matrix{Float64}(I, K+1, K+1))
            # Renormalise β and π rows for new K
            β[K+1] = 1.0 - sum(β[1:K]); β .= β ./ sum(β)
            for j in 1:K+1
                π_vec[j][K+1] = 1.0 - sum(π_vec[j][1:K])
                π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
            end
        end

        # Update (μ, Σ) and ν
        for k in 1:K
            idx     = findall(==(k), S)
            Y_k     = Y[idx, :]
            τ_k     = τ[idx]
            Tau     = sum(τ_k)
            Lambda  = vec(sum(τ_k .* Y_k, dims=1))
            Y_bar   = Lambda / Tau
            diff_mu = Y_bar .- mu0
            centered = Y_k .- Y_bar'
            S_k      = centered' * (centered .* τ_k)
            
            mu_k    = (kappa0*mu0 .+ Lambda) ./ (kappa0 + Tau)
            kappa_k = kappa0 + Tau
            rho_k   = rho0   + m[k]
            Psi_k   = safe_pdmat(Psi0 + S_k +
                      (kappa0*Tau)/(kappa0+Tau) * (diff_mu * diff_mu'))
            
            Σ_mat   = rand(InverseWishart(rho_k, Psi_k))
            Σ[k]    = safe_pdmat(Σ_mat)
            μ[k]    = rand(MvNormal(mu_k, Σ[k] / kappa_k))
        end
        Σ_mat     = rand(InverseWishart(rho0, Psi0))
        Σ[K+1]    = safe_pdmat(Σ_mat)
        μ[K+1]    = rand(MvNormal(mu0, Σ[K+1] / kappa0))

        # MH Update for ν
        sum_Tau = sum(τ)
        log_Tau = sum(log.(τ))

        iota_prop = rand(Normal(ι, mh_walk))
        nu_prop   = 2 + exp(iota_prop)

        nu_div_2      = ν / 2
        nu_prop_div_2 = nu_prop / 2

        log_likelihood = T * nu_div_2 * log(nu_div_2) - T * loggamma(nu_div_2) + (nu_div_2 - 1) * log_Tau - nu_div_2 * sum_Tau
        log_like_prop  = T * nu_prop_div_2 * log(nu_prop_div_2) - T * loggamma(nu_prop_div_2) + (nu_prop_div_2 - 1) * log_Tau - nu_prop_div_2 * sum_Tau

        log_prior      = - (ι - I1)^2 / (2 * I2^2)
        log_prior_prop = - (iota_prop - I1)^2 / (2 * I2^2)

        if n_iter > N_warmup
            total_prop += 1
        end
        acceptance = exp(min(0, log_like_prop + log_prior_prop - log_likelihood - log_prior))
        if rand() < acceptance
            ι = iota_prop
            ν = nu_prop
            if n_iter > N_warmup
                total_accepted += 1
            end
        end

        # Form CRP variables
        for k in 1:K
            for j in 1:K
                if n_mat[j, k] == 0
                    q_mat[j, k] = 0
                else
                    base = α[j] * β[k] + λ[j] * δ[j, k]
                    b     = zeros(Int, n_mat[j, k])
                    b[1]  = 1
                    for l in 2:n_mat[j, k]
                        b[l] = rand(safe_bernoulli(base / (base + l - 1)))
                    end
                    q_mat[j, k] = sum(b)
                end
            end
        end
        q_alpha_diag  = zeros(Int, K)
        q_lambda_diag = zeros(Int, K)
        for k in 1:K
            qkk = q_mat[k, k]
            if qkk > 0
                a = α[k]*β[k]; b = λ[k]
                denom = a + b
                pα = denom > 0 ? clamp(a/denom, 0.0, 1.0) : 0.5
                q_alpha_diag[k]  = rand(Binomial(qkk, pα))
                q_lambda_diag[k] = qkk - q_alpha_diag[k]
            end
        end
        for k in 1:K
            Q[k] = (sum(q_mat[:, k]) - q_mat[k, k]) + q_alpha_diag[k]
        end
        Q[S[1]] += 1
        M = sum(Q)

        # Update γ
        r = rand(safe_beta(γ + 1, M))
        numerator = G1 + K - 1
        denominator = numerator + M * (G2 - safe_log(r))
        z = rand(safe_bernoulli(numerator/denominator))
        γ = rand(safe_gamma(numerator + z, G2 - safe_log(r)))

        # Update β
        β = safe_dirichlet(vcat(Q, γ))
        β .= β ./ sum(β)

        # Update ξ and χ
        for j in 1:K
            ξ[j] = rand(safe_beta(X1 + q_lambda_diag[j], X2 + sum(q_mat[j, :]) - q_lambda_diag[j]))
            r_j  = rand(safe_beta(χ[j] + 1, sum(n_mat[j, :])))
            numerator = C1 + sum(q_mat[j, :]) - 1
            denominator = numerator + sum(n_mat[j, :]) * (C2 - safe_log(r_j))
            z_j  = rand(safe_bernoulli(numerator/denominator))
            χ[j] = rand(safe_gamma(numerator + z_j, C2 - safe_log(r_j)))
            α[j] = (1 - ξ[j]) * χ[j]
            λ[j] = ξ[j]  * χ[j]
        end
        ξ[K+1] = rand(safe_beta(X1, X2))
        χ[K+1] = rand(safe_gamma(C1, C2))
        α[K+1] = (1 - ξ[K+1]) * χ[K+1]
        λ[K+1] = ξ[K+1]  * χ[K+1]

        # Update π
        for j in 1:K+1
            π_vec[j] = safe_dirichlet(α[j] .* β .+ λ[j] .* δ[j, :] .+ n_mat[j, :])
            π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
        end 

        if n_iter > N_warmup

            S_future = zeros(Int, H)
            S_future[1] = sample(1:K+1, Weights(π_vec[S[T]]))
            for h in 2:H
                S_future[h] = sample(1:K+1, Weights(π_vec[S_future[h-1]]))
            end

            for h in 1:H
                k = S_future[h]
                if m[k] > 0
                    idx     = findall(==(k), S)
                    Y_k     = Y[idx, :]
                    τ_k     = τ[idx]
                    Tau     = sum(τ_k)
                    Lambda  = vec(sum(τ_k .* Y_k, dims=1))
                    Y_bar   = Lambda / Tau
                    diff_mu = Y_bar .- mu0
                    centered = Y_k .- Y_bar'
                    S_k      = centered' * (centered .* τ_k)
                    
                    mu_k    = (kappa0*mu0 .+ Lambda) ./ (kappa0 + Tau)
                    kappa_k = kappa0 + Tau
                    rho_k   = rho0   + m[k]
                    Psi_k   = safe_pdmat(Psi0 + S_k +
                              (kappa0*Tau)/(kappa0+Tau) * (diff_mu * diff_mu'))
                    
                    Σ_mat   = rand(InverseWishart(rho_k, Psi_k))
                    Σ_h    = safe_pdmat(Σ_mat)
                    μ_h    = rand(MvNormal(mu_k, Σ_h / kappa_k))

                    # MH Update for ν
                    sum_Tau = sum(τ)
                    log_Tau = sum(log.(τ))

                    iota_prop = rand(Normal(ι, mh_walk))
                    nu_prop   = 2 + exp(iota_prop)

                    nu_div_2      = ν / 2
                    nu_prop_div_2 = nu_prop / 2

                    log_likelihood = T * nu_div_2 * log(nu_div_2) - T * loggamma(nu_div_2) + (nu_div_2 - 1) * log_Tau - nu_div_2 * sum_Tau
                    log_like_prop  = T * nu_prop_div_2 * log(nu_prop_div_2) - T * loggamma(nu_prop_div_2) + (nu_prop_div_2 - 1) * log_Tau - nu_prop_div_2 * sum_Tau

                    log_prior      = - (ι - I1)^2 / (2 * I2^2)
                    log_prior_prop = - (iota_prop - I1)^2 / (2 * I2^2)

                    acceptance = exp(min(0, log_like_prop + log_prior_prop - log_likelihood - log_prior))
                    if rand() < acceptance
                        ι_h = iota_prop
                        ν_h = nu_prop
                    else
                        ι_h = ι
                        ν_h = ν
                    end
                else
                    Σ_mat   = rand(InverseWishart(rho0, Psi0))
                    Σ_h     = safe_pdmat(Σ_mat)
                    μ_h     = rand(MvNormal(mu0, Σ_h / kappa0))
                    ι_h = rand(Normal(I1, I2))
                    ν_h = 2 + exp(ι_h)
                end

                # Build tail-weighted risk matrix
                cov_h = Σ_h * (ν_h / (ν_h - 2))
                sd     = sqrt.(diag(cov_h))
                Dinv   = Diagonal(1.0 ./ clamp.(sd, 1e-12, Inf))
                R_h    = Symmetric(Dinv * cov_h * Dinv)
                L = fill(0.0, D, D)
                ν1 = ν_h + 1.0
                for j in 1:D-1, i in (j+1):D
                    ρ = clamp(R_h[i,j], -1 + 10e-4, 1 - 10e-4)
                    a = sqrt(((ν1) * (1 - ρ)) / (1 + ρ))
                    L[i,j] = L[j,i] = 2 * cdf(TDist(ν1), -a)
                end
                Λ_h = Symmetric(L)
                M_h = cov_h .* Λ_h

                if n_iter == N_warmup + 1
                    mu_bar[h, :]       = μ_h
                    mumuT_bar[h, :, :] = μ_h * μ_h'
                    Sigma_bar[h, :, :] = cov_h
                    M_bar[h, :, :]     = M_h
                else
                    mu_bar[h, :]       += μ_h
                    mumuT_bar[h, :, :] += μ_h * μ_h'
                    Sigma_bar[h, :, :] += cov_h
                    M_bar[h, :, :]     += M_h
                end
            end
        end
    end

    mu_bar    ./= N_sample
    EΣ        = Sigma_bar    ./ N_sample
    EmumuT    = mumuT_bar    ./ N_sample
    M_bar     = M_bar        ./ N_sample
    Covμ      = zeros(H, D, D)
    for h in 1:H
        Covμ[h, :, :] = EmumuT[h, :, :] .- mu_bar[h, :]*mu_bar[h, :]' 
    end
    fullΣ = EΣ .+ Covμ

    # Package up the final sampler state for warm-starting the next day
    state_out = PPState(;
        K = K,
        S = copy(S),
        γ = γ,
        β = copy(β),
        ξ = copy(ξ),
        χ = copy(χ),
        α = copy(α),
        λ = copy(λ),
        π_vec = deepcopy(π_vec),
        μ = deepcopy(μ),
        Σ = deepcopy(Σ),
        τ = deepcopy(τ),
        ι = ι,
        ν = ν,
    )

    return (mu_bar = mu_bar, Sigma_bar = fullΣ, M_bar = M_bar, K = K, state = state_out)
end


run_pp (generic function with 1 method)

In [12]:
# Dimensions
D      = size(Y, 2)
T      = lookback
H      = 20
n_days = size(Y_test, 1)

# Starting portfolio state
x = fill(1/D, D)
h = fill(1000/D, D)
V = [sum(h)]
days_traded = 0

# Drawdown / risk settings and costs
gamma0   = 4.0
drawdown = 60
D_max    = 0.08
kappa    = fill(0.001, D)

# Per-test CSV (overwrite header each run)
test_csv = "nu-NP.csv"
open(test_csv, "w") do io
    hdr = ["iter", "final_K", "fees", "V_new", "C", "SLB", "MMM", "APD", "WIT", "WMT", "MCD", "AEP", "UNH", "TRI"]
    println(io, join(hdr, ","))
end

pp_state = nothing

for day in 1:n_days
    # Run sampler
    pp = run_pp(Matrix(Y), T, H, D; state=pp_state)

    mu_bar, Sigma_bar, M_bar, K = pp.mu_bar, pp.Sigma_bar, pp.M_bar, pp.K
    pp_state = pp.state

    # Drawdown-aware risk aversion
    epsM = 1e-12
    function dd_gamma(M_at, V_t; gamma0, D_max)
        D_rt = (M_at - V_t) / max(M_at, epsM)
        return gamma0 * D_max / max(D_max - D_rt, 1e-5)
    end

    gamma_vec = zeros(H)
    nV = length(V)
    for t in 1:H
        idx = clamp(nV - (t - 1), 1, nV)
        s   = max(1, idx - drawdown + 1)
        M_at = maximum(@view V[s:idx])
        gamma_vec[t] = dd_gamma(M_at, V[idx]; gamma0=gamma0, D_max=D_max)
    end

    prev = copy(x)

    # Two-step optimisation
    # Build risk tensors
    Theta_true = [Sigma_bar[t, :, :] .+ M_bar[t, :, :] for t in 1:H]
    Theta_psd  = [Sigma_bar[t, :, :] for t in 1:H]
    
    # Convex first solve
    function solve_model_simple(Theta::Vector{Matrix{Float64}}; 
                                X_start::Union{Nothing,Matrix{Float64}}=nothing,
                                fee_cap_bps::Float64 = 2.5,
                                cap_all_h::Bool = false)
        model = Model(Ipopt.Optimizer)
        set_silent(model)

        @variable(model, -0.3 <= x_var[1:D, 1:H] <= 0.5)
        @constraint(model, [t=1:H], sum(x_var[i,t] for i in 1:D) == 1)

        # Convex turnover variables
        @variable(model, y[1:D, 1:H] >= 0)

        # t = 1
        @constraint(model, [i=1:D], y[i,1] >=  x_var[i,1] - prev[i])
        @constraint(model, [i=1:D], y[i,1] >= -x_var[i,1] + prev[i])

        # t ≥ 2
        @constraint(model, [i=1:D, t=2:H], y[i,t] >=  x_var[i,t] - x_var[i,t-1])
        @constraint(model, [i=1:D, t=2:H], y[i,t] >= -x_var[i,t] + x_var[i,t-1])

        # Daily fee cap
        fee_cap = fee_cap_bps / 10000.0
        if cap_all_h
            @constraint(model, [t=1:H], sum(kappa[i] * y[i,t] for i in 1:D) <= fee_cap + 1e-10)
        else
            @constraint(model, sum(kappa[i] * y[i,1] for i in 1:D) <= fee_cap + 1e-10)
        end

        # Objective
        @expression(model, return_term, sum(mu_bar[t,i] * x_var[i,t] for t in 1:H, i in 1:D))
        @expression(model, risk_term,   sum(gamma_vec[t] * sum(Theta[t][i,j] * x_var[i,t] * x_var[j,t]
                                            for i in 1:D, j in 1:D) for t in 1:H))
        @expression(model, tc_term,     sum(kappa[i] * y[i,t] for i in 1:D, t in 1:H))
        @objective(model, Max, return_term - risk_term - tc_term)

        if X_start !== nothing
            for t in 1:H, i in 1:D
                set_start_value(x_var[i,t], X_start[i,t])
            end
        end

        optimize!(model)
        st = termination_status(model)
        if !(st in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED))
            return hcat([t == 1 ? copy(prev) : copy(prev) for t in 1:H]...)
        end
        sol = value.(x_var)
        hcat([sol[:,t] for t in 1:H]...)
    end

    # Full second solve
    function solve_model_full(Theta::Vector{Matrix{Float64}}; 
                            X_start::Union{Nothing,Matrix{Float64}}=nothing,
                            fee_cap_bps::Float64 = 2.5,
                            cap_all_h::Bool = false)
        model = Model(Ipopt.Optimizer)
        set_silent(model)

        @variable(model, -0.3 <= x_var[1:D, 1:H] <= 0.5)
        @constraint(model, [t=1:H], sum(x_var[i,t] for i in 1:D) == 1)

        @variable(model, y[1:D, 1:H] >= 0)

        # t = 1
        @constraint(model, [i=1:D], y[i,1] >=  x_var[i,1] - prev[i])
        @constraint(model, [i=1:D], y[i,1] >= -x_var[i,1] + prev[i])

        # t ≥ 2
        @expression(model, denom[t=2:H], sum(x_var[j,t-1] * (1 + mu_bar[t-1,j]) for j in 1:D))
        @variable(model, w[1:D, 2:H])
        @constraint(model, [t=2:H], sum(w[i,t] for i in 1:D) == 1)
        @NLconstraint(model, [i=1:D, t=2:H], w[i,t] * denom[t] == x_var[i,t-1] * (1 + mu_bar[t-1,i]))
        @constraint(model, [i=1:D, t=2:H], y[i,t] >=  x_var[i,t] - w[i,t])
        @constraint(model, [i=1:D, t=2:H], y[i,t] >= -x_var[i,t] + w[i,t])

        # Daily fee cap
        fee_cap = fee_cap_bps / 10000.0
        if cap_all_h
            @constraint(model, [t=1:H], sum(kappa[i] * y[i,t] for i in 1:D) <= fee_cap + 1e-10)
        else
            @constraint(model, sum(kappa[i] * y[i,1] for i in 1:D) <= fee_cap + 1e-10)
        end

        # Objective
        @expression(model, return_term, sum(mu_bar[t,i] * x_var[i,t] for t in 1:H, i in 1:D))
        @expression(model, risk_term,   sum(gamma_vec[t] * sum(Theta[t][i,j] * x_var[i,t] * x_var[j,t]
                                            for i in 1:D, j in 1:D) for t in 1:H))
        @expression(model, tc_term,     sum(kappa[i] * y[i,t] for i in 1:D, t in 1:H))
        @objective(model, Max, return_term - risk_term - tc_term)

        if X_start !== nothing
            for t in 1:H, i in 1:D
                set_start_value(x_var[i,t], X_start[i,t])
            end
        end

        optimize!(model)
        st = termination_status(model)
        if !(st in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED))
            return hcat([t == 1 ? copy(prev) : copy(prev) for t in 1:H]...)
        end
        sol = value.(x_var)
        hcat([sol[:,t] for t in 1:H]...)
    end

    # Pick the better under the true objective (in case second solve gave worse solution)
    best_X(X_init, X_opt, Theta_true; mu_bar, gamma_vec, kappa, prev) = begin
        D, H = size(X_init)
        wprev(X, t) = (d = X[:,t-1] .* (1 .+ mu_bar[t-1,:]); d ./ max(sum(d), eps()))
        f = X -> begin
            ret  = sum(mu_bar[t,:]' * X[:,t]                             for t in 1:H)
            risk = sum(gamma_vec[t] * (X[:,t]' * Theta_true[t] * X[:,t]) for t in 1:H)
            tc   = sum(kappa .* abs.(X[:,1] .- prev)) +
                sum(sum(kappa .* abs.(X[:,t] .- wprev(X,t)))          for t in 2:H)
            ret - risk - tc
        end
        f(X_opt) ≥ f(X_init) ? X_opt : X_init
    end

    X_init = solve_model_simple(Theta_psd;  fee_cap_bps=2.5, cap_all_h=false)
    X_opt  = solve_model_full(  Theta_true; X_start=X_init, fee_cap_bps=2.5, cap_all_h=false)
    X_final = best_X(X_init, X_opt, Theta_true; mu_bar=mu_bar, gamma_vec=gamma_vec, kappa=kappa, prev=prev)
    x_target = X_final[:,1]
    
    # Rebalance with costs applied before returns
    V_pre    = sum(h)
    h_raw    = x_target * V_pre
    total_tc = sum(kappa .* abs.(h_raw .- h))
    V_post   = V_pre - total_tc
    
    h = x_target * V_post
    x = h / V_post
    
    # Apply next-day simple returns
    r_next = vec(Y_test[1, :])
    h      .= h .* (1 .+ r_next)
    
    V_new   = sum(h)
    x      .= h ./ V_new
    
    # Roll Y / Y_test
    Y      = vcat(Y[2:end, :], Y_test[1:1, :])
    Y_test = Y_test[2:end, :]
    
    # Record
    days_traded += 1
    push!(V, V_new)

    row = vcat([days_traded, K, total_tc, V_new], x_target)
    open(test_csv, "a") do io
        println(io, join(row, ","))
    end
end